<a href="https://colab.research.google.com/github/Onedory/DDos-detection-LLM/blob/main/11_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A6%9D%EA%B0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install imbalanced-learn


In [ ]:
# 로짓 경로
bert_logits_path = '/content/drive/MyDrive/LLM/logits-확률평균/bert_logits_val.pkl'
roberta_logits_path = '/content/drive/MyDrive/LLM/logits-확률평균/roberta_logits_val.pkl'

# 레이블 경로
train_labels_path = '/content/drive/MyDrive/LLM/data/train_labels.pkl'

import pickle
# 데이터 로드 함수
def load_data(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

# 로짓 및 레이블 데이터 불러오기
bert_logits = load_data(bert_logits_path)
roberta_logits = load_data(roberta_logits_path)
train_labels = load_data(train_labels_path)

# 데이터 크기 확인
print(f"BERT logits shape: {bert_logits.shape}")
print(f"RoBERTa logits shape: {roberta_logits.shape}")
print(f"Train labels length: {len(train_labels)}")


BERT logits shape: (566149, 2)
RoBERTa logits shape: (566149, 2)
Train labels length: 2264594


In [ ]:
# 로짓과 레이블 크기 출력
print(f"bert_logits shape: {bert_logits.shape}")
print(f"train_labels length: {len(train_labels)}")


bert_logits shape: (566149, 2)
train_labels length: 2264594


In [ ]:
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from collections import Counter

# 데이터 준비
X = np.hstack([bert_logits, roberta_logits])
y = np.array(train_labels)

# 훈련/검증 데이터 나누기
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# 기존 SMOTE
smote = SMOTE(random_state=42)
X_resampled_smote, y_resampled_smote = smote.fit_resample(X_train, y_train)
print("SMOTE Class Distribution:", Counter(y_resampled_smote))

# Borderline-SMOTE
borderline_smote = BorderlineSMOTE(random_state=42)
X_resampled_borderline, y_resampled_borderline = borderline_smote.fit_resample(X_train, y_train)
print("Borderline-SMOTE Class Distribution:", Counter(y_resampled_borderline))

# 모델 학습 및 평가
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# 기존 SMOTE 모델
rf_model.fit(X_resampled_smote, y_resampled_smote)
y_pred_smote = rf_model.predict(X_valid)
print("\nSMOTE Performance:")
print(f"Precision: {precision_score(y_valid, y_pred_smote, average='weighted')}")
print(f"Recall: {recall_score(y_valid, y_pred_smote, average='weighted')}")
print(f"F1-score: {f1_score(y_valid, y_pred_smote, average='weighted')}")
print(f"Confusion Matrix:\n{confusion_matrix(y_valid, y_pred_smote)}")

# Borderline-SMOTE 모델
rf_model.fit(X_resampled_borderline, y_resampled_borderline)
y_pred_borderline = rf_model.predict(X_valid)
print("\nBorderline-SMOTE Performance:")
print(f"Precision: {precision_score(y_valid, y_pred_borderline, average='weighted')}")
print(f"Recall: {recall_score(y_valid, y_pred_borderline, average='weighted')}")
print(f"F1-score: {f1_score(y_valid, y_pred_borderline, average='weighted')}")
print(f"Confusion Matrix:\n{confusion_matrix(y_valid, y_pred_borderline)}")


SMOTE Class Distribution: Counter({0: 364050, 1: 364050})
Borderline-SMOTE Class Distribution: Counter({0: 364050, 1: 364050})

SMOTE Performance:
Precision: 0.6846548664459403
Recall: 0.6473107833613
F1-score: 0.6641971073888064
Confusion Matrix:
[[67622 23439]
 [16496  5673]]

Borderline-SMOTE Performance:
Precision: 0.6856127501669647
Recall: 0.6542877329329683
F1-score: 0.6686731107372504
Confusion Matrix:
[[68570 22491]
 [16654  5515]]


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
import numpy as np

# SMOTE와 Random Forest 모델을 결합하는 함수 정의
def smote_rf_pipeline(X_train, y_train):
    smote = SMOTE(random_state=42)
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

    # SMOTE의 파라미터 설정
    smote_params = {
        'smote__k_neighbors': [3, 5, 7, 10, 15],
        'smote__sampling_strategy': ['auto', 0.5, 0.75, 1.0],
    }

    # Pipeline을 사용하여 SMOTE와 모델을 결합
    from imblearn.pipeline import Pipeline
    pipeline = Pipeline([
        ('smote', smote),
        ('rf', rf_model)
    ])

    # 평가 함수 정의
    scorer = make_scorer(accuracy_score)

    # RandomizedSearchCV를 사용하여 하이퍼파라미터 최적화
    random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=smote_params, scoring=scorer, n_iter=10, cv=5, n_jobs=-1, random_state=42)

    random_search.fit(X_train, y_train)

    return random_search.best_params_, random_search.best_score_

# 최적 파라미터 찾기
best_params, best_score = smote_rf_pipeline(X_train, y_train)

print(f"Best Parameters: {best_params}")
print(f"Best Accuracy: {best_score}")


Best Parameters: {'smote__sampling_strategy': 0.5, 'smote__k_neighbors': 3}
Best Accuracy: 0.7523685260118746


SMOTE 데이터증강

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from collections import Counter
import joblib

# 모델 불러오기
model_path = '/content/drive/MyDrive/LLM/random_forest_model.joblib'
rf_model = joblib.load(model_path)

# SMOTE 데이터 증강
def smote_augmentation(X_train, y_train, sampling_strategy=0.5, k_neighbors=5):
    smote = SMOTE(random_state=42, sampling_strategy=sampling_strategy, k_neighbors=k_neighbors)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    print(f"Before SMOTE: {Counter(y_train)}")
    print(f"After SMOTE: {Counter(y_resampled)}")
    return X_resampled, y_resampled

# SMOTE를 활용한 학습 결과 출력
def train_and_evaluate_with_smote(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5, k_neighbors=5):
    # 데이터 증강
    X_resampled, y_resampled = smote_augmentation(X_train, y_train, sampling_strategy, k_neighbors)


    rf_model.fit(X_resampled, y_resampled)

    # 검증 데이터로 성능 평가
    y_pred = rf_model.predict(X_valid)

    # 성능 평가
    accuracy = accuracy_score(y_valid, y_pred)
    precision = precision_score(y_valid, y_pred, average='weighted')
    recall = recall_score(y_valid, y_pred, average='weighted')
    f1 = f1_score(y_valid, y_pred, average='weighted')
    cm = confusion_matrix(y_valid, y_pred)

    # 결과 출력
    print("\nSMOTE Results (with training):")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    print(f"Confusion Matrix:\n{cm}")

# 예시 실행
train_and_evaluate_with_smote(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5, k_neighbors=5)


Before SMOTE: Counter({0: 364050, 1: 88869})
After SMOTE: Counter({0: 364050, 1: 182025})

SMOTE Results (with training):
Accuracy: 0.7508522476375519
Precision: 0.6831993168037969
Recall: 0.7508522476375519
F1-score: 0.7101328724812705
Confusion Matrix:
[[83186  7875]
 [20336  1833]]


Borderline-SMOTE

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from collections import Counter
import joblib
from sklearn.ensemble import RandomForestClassifier

# 모델 불러오기
model_path = '/content/drive/MyDrive/LLM/random_forest_model.joblib'
rf_model = joblib.load(model_path)

# Borderline-SMOTE 데이터 증강
def borderline_smote_augmentation(X_train, y_train, sampling_strategy=0.5, k_neighbors=5):
    smote = BorderlineSMOTE(random_state=42, sampling_strategy=sampling_strategy, k_neighbors=k_neighbors)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    print(f"Before Borderline-SMOTE: {Counter(y_train)}")
    print(f"After Borderline-SMOTE: {Counter(y_resampled)}")
    return X_resampled, y_resampled

# Borderline-SMOTE를 활용한 학습 결과 출력
def train_and_evaluate_with_borderline_smote(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5, k_neighbors=5):
    # 데이터 증강
    X_resampled, y_resampled = borderline_smote_augmentation(X_train, y_train, sampling_strategy, k_neighbors)

    # 증강된 데이터로 모델 재학습
    rf_model.fit(X_resampled, y_resampled)

    # 검증 데이터로 성능 평가
    y_pred = rf_model.predict(X_valid)

    # 성능 평가
    accuracy = accuracy_score(y_valid, y_pred)
    precision = precision_score(y_valid, y_pred, average='weighted')
    recall = recall_score(y_valid, y_pred, average='weighted')
    f1 = f1_score(y_valid, y_pred, average='weighted')
    cm = confusion_matrix(y_valid, y_pred)

    # 결과 출력
    print("\nBorderline-SMOTE Results (with training):")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    print(f"Confusion Matrix:\n{cm}")

# 예시 실행
train_and_evaluate_with_borderline_smote(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5, k_neighbors=5)


Before Borderline-SMOTE: Counter({0: 364050, 1: 88869})
After Borderline-SMOTE: Counter({0: 364050, 1: 182025})

Borderline-SMOTE Results (with training):
Accuracy: 0.7506491212576173
Precision: 0.683729033987498
Recall: 0.7506491212576173
F1-score: 0.7103740443247817
Confusion Matrix:
[[83125  7936]
 [20298  1871]]


 ADASYN (Adaptive Synthetic Sampling Approach for Imbalanced Learning)

In [ ]:
from imblearn.over_sampling import ADASYN

# 모델 불러오기
model_path = '/content/drive/MyDrive/LLM/random_forest_model.joblib'
rf_model = joblib.load(model_path)

def adasyn_augmentation(X_train, y_train, sampling_strategy=0.5, n_neighbors=5):
    adasyn = ADASYN(random_state=42, sampling_strategy=sampling_strategy, n_neighbors=n_neighbors)
    X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)
    print(f"Before ADASYN: {Counter(y_train)}")
    print(f"After ADASYN: {Counter(y_resampled)}")
    return X_resampled, y_resampled

def train_and_evaluate_with_adasyn(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5, n_neighbors=5):
    X_resampled, y_resampled = adasyn_augmentation(X_train, y_train, sampling_strategy, n_neighbors)

    # 모델 학습
    rf_model.fit(X_resampled, y_resampled)

    # 성능 평가
    y_pred = rf_model.predict(X_valid)

    accuracy = accuracy_score(y_valid, y_pred)
    precision = precision_score(y_valid, y_pred, average='weighted')
    recall = recall_score(y_valid, y_pred, average='weighted')
    f1 = f1_score(y_valid, y_pred, average='weighted')
    cm = confusion_matrix(y_valid, y_pred)

    # 결과 출력
    print("\nADASYN Results (with training):")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    print(f"Confusion Matrix:\n{cm}")

# 예시 실행
train_and_evaluate_with_adasyn(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5, n_neighbors=5)


Before ADASYN: Counter({0: 364050, 1: 88869})
After ADASYN: Counter({0: 364050, 1: 177120})

ADASYN Results (with training):
Accuracy: 0.7527333745473814
Precision: 0.683565989189825
Recall: 0.7527333745473814
F1-score: 0.7108511102904942
Confusion Matrix:
[[83444  7617]
 [20381  1788]]


3. SMOTEENN (SMOTE + Edited Nearest Neighbors)

In [ ]:
from imblearn.combine import SMOTEENN


# 모델 불러오기
model_path = '/content/drive/MyDrive/LLM/random_forest_model.joblib'
rf_model = joblib.load(model_path)
def smoteenn_augmentation(X_train, y_train, sampling_strategy=0.5):
    smoteenn = SMOTEENN(random_state=42, sampling_strategy=sampling_strategy)
    X_resampled, y_resampled = smoteenn.fit_resample(X_train, y_train)
    print(f"Before SMOTEENN: {Counter(y_train)}")
    print(f"After SMOTEENN: {Counter(y_resampled)}")
    return X_resampled, y_resampled

def train_and_evaluate_with_smoteenn(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5):
    X_resampled, y_resampled = smoteenn_augmentation(X_train, y_train, sampling_strategy)

    # 모델 학습
    rf_model.fit(X_resampled, y_resampled)

    # 성능 평가
    y_pred = rf_model.predict(X_valid)

    accuracy = accuracy_score(y_valid, y_pred)
    precision = precision_score(y_valid, y_pred, average='weighted')
    recall = recall_score(y_valid, y_pred, average='weighted')
    f1 = f1_score(y_valid, y_pred, average='weighted')
    cm = confusion_matrix(y_valid, y_pred)

    # 결과 출력
    print("\nSMOTEENN Results (with training):")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    print(f"Confusion Matrix:\n{cm}")

# 예시 실행
train_and_evaluate_with_smoteenn(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5)


Before SMOTEENN: Counter({0: 364050, 1: 88869})
After SMOTEENN: Counter({0: 174111, 1: 50053})

SMOTEENN Results (with training):
Accuracy: 0.7708469486885101
Precision: 0.6862232754913581
Recall: 0.7708469486885101
F1-score: 0.71619819319471
Confusion Matrix:
[[86019  5042]
 [20905  1264]]


4. SMOTETomek (SMOTE + Tomek Links)

In [ ]:
from imblearn.combine import SMOTETomek

# 모델 불러오기
model_path = '/content/drive/MyDrive/LLM/random_forest_model.joblib'
rf_model = joblib.load(model_path)


def smotetomek_augmentation(X_train, y_train, sampling_strategy=0.5):
    smotetomek = SMOTETomek(random_state=42, sampling_strategy=sampling_strategy)
    X_resampled, y_resampled = smotetomek.fit_resample(X_train, y_train)
    print(f"Before SMOTETomek: {Counter(y_train)}")
    print(f"After SMOTETomek: {Counter(y_resampled)}")
    return X_resampled, y_resampled

def train_and_evaluate_with_smotetomek(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5):
    X_resampled, y_resampled = smotetomek_augmentation(X_train, y_train, sampling_strategy)

    # 모델 학습
    rf_model.fit(X_resampled, y_resampled)

    # 성능 평가
    y_pred = rf_model.predict(X_valid)

    accuracy = accuracy_score(y_valid, y_pred)
    precision = precision_score(y_valid, y_pred, average='weighted')
    recall = recall_score(y_valid, y_pred, average='weighted')
    f1 = f1_score(y_valid, y_pred, average='weighted')
    cm = confusion_matrix(y_valid, y_pred)

    # 결과 출력
    print("\nSMOTETomek Results (with training):")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    print(f"Confusion Matrix:\n{cm}")

# 예시 실행
train_and_evaluate_with_smotetomek(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5)


Before SMOTETomek: Counter({0: 364050, 1: 88869})
After SMOTETomek: Counter({0: 343568, 1: 161543})

SMOTETomek Results (with training):
Accuracy: 0.7556919544290382
Precision: 0.6832741551786828
Recall: 0.7556919544290382
F1-score: 0.7114648187972993
Confusion Matrix:
[[83899  7162]
 [20501  1668]]


5. Random Under-sampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler


# 모델 불러오기
model_path = '/content/drive/MyDrive/LLM/random_forest_model.joblib'
rf_model = joblib.load(model_path)

def under_sampling_augmentation(X_train, y_train, sampling_strategy=0.5):
    under_sampler = RandomUnderSampler(random_state=42, sampling_strategy=sampling_strategy)
    X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)
    print(f"Before Under-sampling: {Counter(y_train)}")
    print(f"After Under-sampling: {Counter(y_resampled)}")
    return X_resampled, y_resampled

def train_and_evaluate_with_under_sampling(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5):
    X_resampled, y_resampled = under_sampling_augmentation(X_train, y_train, sampling_strategy)

    # 모델 학습
    rf_model.fit(X_resampled, y_resampled)

    # 성능 평가
    y_pred = rf_model.predict(X_valid)

    accuracy = accuracy_score(y_valid, y_pred)
    precision = precision_score(y_valid, y_pred, average='weighted')
    recall = recall_score(y_valid, y_pred, average='weighted')
    f1 = f1_score(y_valid, y_pred, average='weighted')
    cm = confusion_matrix(y_valid, y_pred)

    # 결과 출력
    print("\nRandom Under-sampling Results (with training):")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    print(f"Confusion Matrix:\n{cm}")

# 예시 실행
train_and_evaluate_with_under_sampling(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5)


Before Under-sampling: Counter({0: 364050, 1: 88869})
After Under-sampling: Counter({0: 177738, 1: 88869})

Random Under-sampling Results (with training):
Accuracy: 0.7353793164355736
Precision: 0.6851181955062448
Recall: 0.7353793164355736
F1-score: 0.7064617587973262
Confusion Matrix:
[[80757 10304]
 [19659  2510]]


6. NearMiss (Under-sampling method)

In [ ]:
from imblearn.under_sampling import NearMiss


# 모델 불러오기
model_path = '/content/drive/MyDrive/LLM/random_forest_model.joblib'
rf_model = joblib.load(model_path)

def nearmiss_augmentation(X_train, y_train, sampling_strategy=0.5):
    nearmiss = NearMiss(sampling_strategy=sampling_strategy)
    X_resampled, y_resampled = nearmiss.fit_resample(X_train, y_train)
    print(f"Before NearMiss: {Counter(y_train)}")
    print(f"After NearMiss: {Counter(y_resampled)}")
    return X_resampled, y_resampled

def train_and_evaluate_with_nearmiss(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5):
    X_resampled, y_resampled = nearmiss_augmentation(X_train, y_train, sampling_strategy)

    # 모델 학습
    rf_model.fit(X_resampled, y_resampled)

    # 성능 평가
    y_pred = rf_model.predict(X_valid)

    accuracy = accuracy_score(y_valid, y_pred)
    precision = precision_score(y_valid, y_pred, average='weighted')
    recall = recall_score(y_valid, y_pred, average='weighted')
    f1 = f1_score(y_valid, y_pred, average='weighted')
    cm = confusion_matrix(y_valid, y_pred)

    # 결과 출력
    print("\nNearMiss Results (with training):")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")
    print(f"Confusion Matrix:\n{cm}")

# 예시 실행
train_and_evaluate_with_nearmiss(X_train, y_train, X_valid, y_valid, sampling_strategy=0.5)


Before NearMiss: Counter({0: 364050, 1: 88869})
After NearMiss: Counter({0: 177738, 1: 88869})

NearMiss Results (with training):
Accuracy: 0.5055197385851806
Precision: 0.6849909955628277
Recall: 0.5055197385851806
F1-score: 0.5562208313163571
Confusion Matrix:
[[46366 44695]
 [11295 10874]]
